In [ ]:
from transformers import GPT2LMHeadModel

In [4]:
model_hf = GPT2LMHeadModel.from_pretrained('gpt2')
sd_hf = model_hf.state_dict()

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from datasets import load_dataset
from model import GPT, GPTConfig

In [3]:
import sys

sys.path.append("/home/htkumar/llms/gpt2_karpathy")

In [ ]:
model = GPT(GPTConfig(vocab_size=50304))  # power of 2 is better for the GPUs
model.to("cuda:0")
model = torch.compile(model)
model

In [ ]:
a = torch.load("/home/htkumar/llms/gpt2_karpathy/logs/model_00000.pt")
a.keys()

In [ ]:
a["model"].keys()

In [ ]:
model.load_state_dict(a["model"])

In [ ]:
model.eval()

In [ ]:
a = torch.tensor([[1, 2, 3], [4, 5, 6]])
a.shape

In [ ]:
a[0:]

In [ ]:
a[0]

In [ ]:
a[:, 1:]

In [ ]:
torch.cuda.is_available()

In [ ]:
torch.cuda.is_bf16_supported()

In [ ]:
torch.tril(torch.ones(8, 8)).view(1, 1, 8, 8)

In [ ]:
x = torch.arange(12).view(2, 2, 3)
x.shape

In [ ]:
q, k, v = x.chunk(3, dim=2)
q.shape, k.shape, v.shape

In [ ]:
q, k, v = x.split(1, dim=2)
q.shape, k.shape, v.shape

In [ ]:
module_dict = nn.ModuleDict(
    dict(
        module_1=nn.Linear(10, 10),
        module_2=nn.Linear(10, 10),
    )
)

In [ ]:
module_dict.keys()

In [ ]:
q.shape

In [ ]:
q.shape[::-1]

In [ ]:
a = torch.arange(6).view(2, 3)
a.shape

In [ ]:
a.t().shape

In [ ]:
import tiktoken

In [ ]:
a = torch.arange(12).view(3, 4).double()
a.shape, a.dtype

In [ ]:
a = F.softmax(a, dim=1)
a

In [ ]:
topk_probs, topk_indices = torch.topk(a, 2, dim=1)

In [ ]:
topk_probs

In [ ]:
topk_indices

In [ ]:
b = torch.multinomial(topk_probs, 1)
b

In [ ]:
b.shape

In [ ]:
xcol = torch.gather(topk_indices, -1, b)
xcol

In [ ]:
t = torch.tensor([[1, 2], [3, 4]])
torch.gather(t, 1, torch.tensor([[0, 0], [1, 0]]))

In [ ]:
torch.gather(t, 0, torch.tensor([[0, 0], [1, 0]]))

In [ ]:
a = torch.tensor(1)
a.shape

In [ ]:
a.item()

In [ ]:
a = torch.tensor([1, 2, 3])
len(a)

In [ ]:
import time

t0 = time.time()
t0

In [ ]:
t1 = time.time()

In [ ]:
(t1 - t0)

In [ ]:
time.time??

In [ ]:
import math

In [ ]:
max_lr = 6e-4
min_lr = max_lr * 0.1
warmup_steps = 10
max_steps = 50


def get_lr(it):
    # linear warmup
    if it < warmup_steps:
        return max_lr * (it + 1) / warmup_steps
    elif it > max_steps:
        return min_lr
    # cosine decay
    decay_ratio = (it - warmup_steps) / (max_steps - warmup_steps)
    assert 0 <= decay_ratio <= 1
    coeff = 0.5 * (1.0 + math.cos(math.pi * decay_ratio))
    return min_lr + coeff * (max_lr - min_lr)

In [ ]:
iters = list(range(100))

In [ ]:
lrs = [get_lr(it) for it in iters]

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.style.use("_mpl-gallery")

In [ ]:
plt.plot(iters, lrs, linewidth=2.0)

In [ ]:
import os

os.getcwd()

In [ ]:
from train_gpt2 import GPT, GPTConfig

model = GPT(GPTConfig(vocab_size=50304))

In [ ]:
optimizer = model.configure_optimizers(
    weight_decay=0.1, learning_rate=3e-4, device="cuda"
)

In [ ]:
optimizer.param_groups[0].keys()

In [ ]:
len(optimizer.param_groups[1]["params"])
# same as number of parameters in this group

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4, betas=(0.9, 0.95), eps=1e-9)

In [ ]:
optimizer.param_groups[0]["lr"]

In [ ]:
import inspect

In [ ]:
inspect.signature(torch.optim.AdamW).parameters

In [ ]:
params_dict = {pn: p for pn, p in model.named_parameters()}
params_dict["transformer.wte.weight"].requires_grad
len(params_dict)

In [ ]:
params_dict = {pn: p for pn, p in params_dict.items() if p.requires_grad}
len(params_dict)

In [ ]:
decay_params = [p for n, p in params_dict.items() if p.dim() >= 2]
nondecay_params = [p for n, p in params_dict.items() if p.dim() < 2]
len(decay_params), len(nondecay_params)

In [ ]:
num_decay_params = sum(p.numel() for p in decay_params)
num_nondecay_params = sum(p.numel() for p in nondecay_params)

In [ ]:
print(num_decay_params, num_nondecay_params)

In [ ]:
# fineweb edu dataset processing

In [ ]:
import multiprocessing as mp
import os

import numpy as np
import tiktoken
from datasets import load_dataset
from tqdm import tqdm

In [ ]:
remote_name = "sample-10BT"  # these are 10B gpt2 tokens sampled from the whole dataset

In [ ]:
# fw = load_dataset("HuggingFaceFW/fineweb-edu", name=remote_name, split="train")

In [ ]:
max_value = np.iinfo(np.uint16).max
max_value

In [ ]:
np.iinfo(np.uint16).min

In [ ]:
a = np.array([2**16 - 1])
a.astype(np.uint16)

In [ ]:
os.path.join(os.path.dirname(__file__), "test")

In [ ]:
os.cpu_count() // 2

In [ ]:
loss = torch.tensor(3.1)

In [ ]:
loss.detach()